## Introduction
This notebook is designed to guide you through the process of building a simple retrieval augmented generation application using Langchain, ChromaDB and OpenAI's GPT-4 LLM. 

![](../assets/4-RAG.png)

### RAG: Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) combines the power of language models with external content retrieval to answer questions based on specific content. This approach allows for more accurate and contextually relevant responses by leveraging a database of information.

The process involves several key steps:
- **Content Retrieval and Storage**: First, we ingest and store our content in a searchable format. This involves fetching content from a URL, splitting it into manageable parts, embedding these parts for semantic search, and storing them in a Vector DB.
- **Question Answering**: When a user question is posed, the system retrieves relevant content from the Vector DB and uses it to generate an answer with the LLM.

This notebook will walk you through setting up a RAG system using ChromaDB for content storage and retrieval, and OpenAI's GPT-4 for question answering.

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(dotenv_path=".envrc")

False

# Run basic example from Langchain Vectorstores Chroma page

[Read more here](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

# Basic Example

In [3]:
import rich
from goob_ai import debugger

In [2]:
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("example_data/state_of_the_union.txt")
documents = loader.load()


In [7]:
rich.inspect(loader)

rich.inspect(documents)

╭───────── <class 'langchain_community.document_loaders.text.TextLoader'> ─────────╮
│ Load text file.                                                                  │
│                                                                                  │
│ ╭──────────────────────────────────────────────────────────────────────────────╮ │
│ │ <langchain_community.document_loaders.text.TextLoader object at 0x1062dc700> │ │
│ ╰──────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                  │
│ autodetect_encoding = False                                                      │
│            encoding = None                                                       │
│           file_path = 'example_data/state_of_the_union.txt'                      │
╰──────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── <class 'list'> ────────────────────────────────────────╮
│ Built-in mutable sequence.                                                                     │
│                                                                                                │
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                          │ │
│ │ │   Document(                                                                              │ │
│ │ │   │   page_content='Madam Speaker, Madam Vice President, our First Lady and Seco'+38479, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                         │ │
│ │ │   )                                                                                      │ │
│ │ ]                                                                                          │ │
│ ╰────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                │
│ 36 attribute(s) not shown. Run inspect(inspect) for options.                                   │
╰────────────────────────────────────────────────────────────────────────────────────────────────╯

In [8]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [9]:
rich.inspect(text_splitter)

rich.inspect(docs)

╭───────── <class 'langchain_text_splitters.character.CharacterTextSplitter'> ─────────╮
│ Splitting text that looks at characters.                                             │
│                                                                                      │
│ ╭──────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <langchain_text_splitters.character.CharacterTextSplitter object at 0x12f9d65f0> │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                      │
│ 40 attribute(s) not shown. Run inspect(inspect) for options.                         │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────── <class 'list'> ───────────────────────────────────────╮
│ Built-in mutable sequence.                                                                   │
│                                                                                              │
│ ╭──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                        │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='Madam Speaker, Madam Vice President, our First Lady and Seco'+879, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='Groups of citizens blocking tanks with their bodies. Everyon'+935, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='Putin’s latest attack on Ukraine was premeditated and unprov'+899, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='We are inflicting pain on Russia and supporting the people o'+887, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='And tonight I am announcing that we will join our allies in '+916, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='For that purpose we’ve mobilized American ground forces, air'+891, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='And I’m taking robust action to make sure the pain of our sa'+894, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='We see the unity among leaders of nations and a more unified'+922, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='And so many families are living paycheck to paycheck, strugg'+833, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(      

In [11]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/Users/malcolm/dev/bossjones/goob_ai/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/malcolm/dev/bossjones/goob_ai/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# Persistant saving to disk example

In [13]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# passing a Chroma client into langchain

In [14]:
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embedding_function,
)

print("There are", langchain_chroma._collection.count(), "in the collection")

/Users/malcolm/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:05<00:00, 15.2MiB/s]


There are 3 in the collection


In [2]:
# # NOTE: This is from GP, let's play with this later
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_openai import AzureOpenAIEmbeddings
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_chroma import Chroma
# import bs4

# # Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# # Split the content into manageable chunks for better retrieval.
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

# # Embed the chunks and store them in ChromaDB for efficient retrieval.
# vectorstore = Chroma.from_documents(documents=splits, embedding=AzureOpenAIEmbeddings(azure_deployment=os.environ["AZURE_EMBEDDINGS_DEPLOYMENT"]))

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# import os
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import AzureChatOpenAI
# from langchain_core.output_parsers import StrOutputParser

# # Set up the RAG chain for retrieving and generating answers.
# retriever = vectorstore.as_retriever()
# system_prompt = ("""
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
# Context: {context}
# """)

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{question}"),
#     ]
# )


# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)


# # Initialize the model with our deployment of Azure OpenAI
# model = AzureChatOpenAI(azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"])

# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | model
#     | StrOutputParser()
# )

In [4]:
# question = "What is tool usage?"
# rag_chain.invoke(question)

"Tool usage is the ability to create, modify, and utilize external objects to perform tasks beyond one's physical and cognitive limits. For language models (LLMs), tool usage involves calling external APIs to access additional information, execute code, or retrieve proprietary data. This capability significantly extends the model's functionality by allowing it to interact with external systems and perform complex, multi-step tasks."